[![img/pythonista.png](img/pythonista.png)](https://www.pythonista.io)

## El proyecto *apispec*.

https://flask-marshmallow.readthedocs.io/en/latest/

In [ ]:
pip install flask-apispec

In [ ]:
from flask import Flask, abort, request
from flask_restful import Api, Resource
from flask_sqlalchemy import SQLAlchemy
from flask_marshmallow import Marshmallow, base_fields
from marshmallow.validate import Length, Range, OneOf
from data import carreras
from json import loads

In [ ]:
from flask_apispec import marshal_with
from flask_apispec.views import MethodResource 

In [ ]:
from apispec import APISpec
from apispec.ext.marshmallow import MarshmallowPlugin
from flask_apispec.extension import FlaskApiSpec

In [ ]:
app = Flask(__name__)
app.config["SQLALCHEMY_DATABASE_URI"] = "sqlite:///data/alumnosql.sqlite"
app.config["SQLALCHEMY_TRACK_MODIFICATIONS"] = False

In [ ]:
app.config.update({
    'APISPEC_SPEC': APISpec(
        title='Gestión de alumnos.',
        version='v1',
        plugins=[MarshmallowPlugin()],
        openapi_version='2.0.0'
    ),
    'APISPEC_SWAGGER_URL': '/swagger/',  # URI to access API Doc JSON 
    'APISPEC_SWAGGER_UI_URL': '/swagger-ui/'  # URI to access UI of API Doc
})

In [ ]:
# Order matters: Initialize SQLAlchemy before Marshmallow
api = Api(app)
db = SQLAlchemy(app)
ma = Marshmallow(app)

In [ ]:
class Alumno(db.Model):
    __tablename__ = 'alumno'
    cuenta = db.Column(db.Integer, primary_key=True)
    nombre = db.Column(db.String(50))
    primer_apellido = db.Column(db.String(50))
    segundo_apellido = db.Column(db.String(50))
    carrera = db.Column(db.String(50))
    semestre = db.Column(db.Integer)
    promedio = db.Column(db.Float)
    al_corriente = db.Column(db.Boolean)

In [ ]:
class AlumnoSchema(ma.SQLAlchemySchema):
    
    
    class Meta:
        model = Alumno
        load_instance = True
    
    
    cuenta = base_fields.Int(required=True, validate=Range(min=1000000, max=9999999))
    nombre = base_fields.String(required=True, validate=Length(min=2, max=50))
    primer_apellido = base_fields.String(required=True, validate=Length(min=2, max=50))
    segundo_apellido = base_fields.String(required=False, validate=Length(min=2, max=50))
    carrera = base_fields.String(required=True, validate=OneOf(carreras))
    semestre = base_fields.Int(required=True, validate=Range(min=1, max=50))
    promedio = base_fields.Float(required=True, validate=Range(min=1, max=50))
    al_corriente = base_fields.Bool(required=True)

In [ ]:
schema_multiple = AlumnoSchema(many=True)
schema = AlumnoSchema()

In [ ]:
@marshal_with(schema_multiple)
class MuestraAlumnos(MethodResource, Resource):
    
    
    def get(Resource):
        alumnos = Alumno.query.all()
        return schema_multiple.dump(alumnos)

In [ ]:
@marshal_with(schema)
class ABCAlumno(MethodResource, Resource):
    
    
    def get(self, cuenta):
        alumno = Alumno.query.filter_by(cuenta=cuenta).first()
        if alumno:
            return schema.dump(alumno)
        else:
            abort(404)
    
    
    def delete(self, cuenta):
        alumno = Alumno.query.filter_by(cuenta=cuenta).first()
        if alumno:
            db.session.delete(alumno)
            db.session.commit()
            return schema.dump(alumno)
        else:
            abort(404)
     
    
    def post(self, cuenta):
        alumno = Alumno.query.filter_by(cuenta=cuenta).first()
        if alumno:
            abort(409)
        else:
            respuesta = loads(request.data)
            respuesta["cuenta"] = cuenta
            db.session.add(schema.load(respuesta))
            db.session.commit()
            return respuesta
     
    
    def put(self, cuenta):
        alumno = Alumno.query.filter_by(cuenta=cuenta).first()
        if alumno:
            respuesta = loads(request.data)
            respuesta["cuenta"] = cuenta
            db.session.add(schema.load(respuesta))
            db.session.commit()
            return respuesta
        else:
            abort(404)

In [ ]:
api.add_resource(ABCAlumno, '/api/<string:cuenta>')
api.add_resource(MuestraAlumnos, '/api/')

In [ ]:
docs = FlaskApiSpec(app)
docs.register(ABCAlumno)
docs.register(MuestraAlumnos)

In [ ]:
if __name__ == '__main__':
    app.run(host='0.0.0.0')

<p style="text-align: center"><a rel="license" href="http://creativecommons.org/licenses/by/4.0/"><img alt="Licencia Creative Commons" style="border-width:0" src="https://i.creativecommons.org/l/by/4.0/80x15.png" /></a><br />Esta obra está bajo una <a rel="license" href="http://creativecommons.org/licenses/by/4.0/">Licencia Creative Commons Atribución 4.0 Internacional</a>.</p>
<p style="text-align: center">&copy; José Luis Chiquete Valdivieso. 2021.</p>